In [41]:
import pandas as pd
anime_df = pd.read_csv("myanimelist_normalized.csv")

anime_df.head()

title                  english  \
0                     Cowboy Bebop             Cowboy Bebop   
1  Cowboy Bebop: Tengoku no Tobira  Cowboy Bebop: The Movie   
2                           Trigun                   Trigun   
3               Witch Hunter Robin       Witch Hunter Robin   
4                   Bouken Ou Beet   Beet the Vandel Buster   

                           japanese   type    source  episodes  \
0                         カウボーイビバップ     TV  Original      26.0   
1                    カウボーイビバップ 天国の扉  Movie  Original       1.0   
2                             トライガン     TV     Manga      26.0   
3  Witch Hunter ROBIN (ウイッチハンターロビン)     TV  Original      26.0   
4                            冒険王ビィト     TV     Manga      52.0   

            status        aired_from          aired_to       duration  ...  \
0  Finished Airing  03/04/1998 02:00  24/04/1999 02:00  24 min per ep  ...   
1  Finished Airing  01/09/2001 02:00               NaN    1 hr 55 min  ...   
2  Finished Airing  01/04/1998 02:00  30/09/1998 02:00  24 min per ep  ...   
3  Finished Airing  03/07/2002 02:00  25/12/2002 01:00  25 min per ep  ...   
4  Finished Airing  30/09/2004 02:00  29/09/2005 02:00  23 min per ep  ...   

     year broadcast_day  broadcast_time  \
0  1998.0     Saturdays        01:00:00   
1     NaN           NaN             NaN   
2  1998.0     Thursdays        01:15:00   
3  2002.0    Wednesdays        01:25:00   
4  2004.0     Thursdays        18:30:00   

                                           producers  \
0  Bandai Visual,Victor Entertainment,Audio Plann...   
1                              Sunrise,Bandai Visual   
2                               Victor Entertainment   
3          Bandai Visual,Dentsu,Victor Entertainment   
4                                    TV Tokyo,Dentsu   

                                licensors         studios  \
0                              Funimation         Sunrise   
1  Sony Pictures Entertainment,Funimation           Bones   
2                              Funimation        Madhouse   
3         Funimation,Bandai Entertainment         Sunrise   
4                Illumitoon Entertainment  Toei Animation   

                              genres            themes demographics  \
0        Action,Award Winning,Sci-Fi  Adult Cast,Space          NaN   
1                      Action,Sci-Fi  Adult Cast,Space          NaN   
2            Action,Adventure,Sci-Fi        Adult Cast      Shounen   
3  Action,Drama,Mystery,Supernatural         Detective          NaN   
4           Action,Adventure,Fantasy               NaN      Shounen   

                 title_normalized  
0                    cowboy bebop  
1  cowboy bebop tengoku no tobira  
2                          trigun  
3              witch hunter robin  
4                  bouken ou beet  

[5 rows x 29 columns]

In [42]:
anime_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26541 entries, 0 to 26540
Data columns (total 29 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   title             26541 non-null  object 
 1   english           11870 non-null  object 
 2   japanese          26428 non-null  object 
 3   type              26537 non-null  object 
 4   source            26541 non-null  object 
 5   episodes          26144 non-null  float64
 6   status            26541 non-null  object 
 7   aired_from        26541 non-null  object 
 8   aired_to          10974 non-null  object 
 9   duration          26541 non-null  object 
 10  duration_minutes  26059 non-null  float64
 11  rating            26113 non-null  object 
 12  score             18319 non-null  float64
 13  scored_by         18319 non-null  float64
 14  rank              20158 non-null  float64
 15  popularity        26541 non-null  int64  
 16  members           26541 non-null  int64 

In [43]:
import re

anime_df["english"] = anime_df["english"].str.lower()
anime_df["english"] = anime_df["english"].str.replace(r"[^\w\s\|’'”“‘]", "", regex=True)
anime_df["english"] = anime_df["english"].str.replace(r"\s+", " ", regex=True).str.strip()

anime_df["japanese"] = anime_df["japanese"].str.lower()
anime_df["japanese"] = anime_df["japanese"].str.replace(r"[^\w\s\|’'”“‘]", "", regex=True)
anime_df["japanese"] = anime_df["japanese"].str.replace(r"\s+", " ", regex=True).str.strip()


anime_df[["english", "japanese"]].head(10)


english                        japanese
0            cowboy bebop                       カウボーイビバップ
1  cowboy bebop the movie                  カウボーイビバップ 天国の扉
2                  trigun                           トライガン
3      witch hunter robin  witch hunter robin ウイッチハンターロビン
4  beet the vandel buster                          冒険王ビィト
5                     NaN                        アイシールド21
6        honey and clover                      ハチミツとクローバー
7                     NaN           ハングリーハート wild striker
8                     NaN          頭文字イニシャルd fourth stage
9                 monster                           モンスター

In [ ]:
import pandas as pd
from fuzzywuzzy import process, fuzz

mangadex_df = pd.read_csv("mangadex_optimized.csv")

mangadex_titles = mangadex_df["title_normalized"].dropna().tolist()

def match_titles(row, mangadex_titles):
    title = row.get("title_normalized", "")
    
    if not isinstance(title, str) or title.strip().lower() == "nan" or title.strip() == "":
        return None
    
    match = process.extractOne(title, mangadex_titles, scorer=fuzz.ratio)
    return match[0] if match and match[1] > 85 else None
anime_df["matched_title_normalized"] = anime_df.apply(match_titles, args=(mangadex_titles,), axis=1)

anime_df.head(10)


title                 english  \
0                     Cowboy Bebop            cowboy bebop   
1  Cowboy Bebop: Tengoku no Tobira  cowboy bebop the movie   
2                           Trigun                  trigun   
3               Witch Hunter Robin      witch hunter robin   
4                   Bouken Ou Beet  beet the vandel buster   
5                     Eyeshield 21                     NaN   
6             Hachimitsu to Clover        honey and clover   
7       Hungry Heart: Wild Striker                     NaN   
8           Initial D Fourth Stage                     NaN   
9                          Monster                 monster   

                         japanese   type    source  episodes           status  \
0                       カウボーイビバップ     TV  Original      26.0  Finished Airing   
1                  カウボーイビバップ 天国の扉  Movie  Original       1.0  Finished Airing   
2                           トライガン     TV     Manga      26.0  Finished Airing   
3  witch hunter robin ウイッチハンターロビン     TV  Original      26.0  Finished Airing   
4                          冒険王ビィト     TV     Manga      52.0  Finished Airing   
5                        アイシールド21     TV     Manga     145.0  Finished Airing   
6                      ハチミツとクローバー     TV     Manga      24.0  Finished Airing   
7           ハングリーハート wild striker     TV     Manga      52.0  Finished Airing   
8          頭文字イニシャルd fourth stage     TV     Manga      24.0  Finished Airing   
9                           モンスター     TV     Manga      74.0  Finished Airing   

         aired_from          aired_to       duration  ...  broadcast_day  \
0  03/04/1998 02:00  24/04/1999 02:00  24 min per ep  ...      Saturdays   
1  01/09/2001 02:00               NaN    1 hr 55 min  ...            NaN   
2  01/04/1998 02:00  30/09/1998 02:00  24 min per ep  ...      Thursdays   
3  03/07/2002 02:00  25/12/2002 01:00  25 min per ep  ...     Wednesdays   
4  30/09/2004 02:00  29/09/2005 02:00  23 min per ep  ...      Thursdays   
5  06/04/2005 02:00  19/03/2008 01:00  23 min per ep  ...     Wednesdays   
6  15/04/2005 02:00  27/09/2005 02:00  23 min per ep  ...        Fridays   
7  11/09/2002 02:00  10/09/2003 02:00  23 min per ep  ...            NaN   
8  17/04/2004 02:00  18/02/2006 01:00  27 min per ep  ...            NaN   
9  07/04/2004 02:00  28/09/2005 02:00  24 min per ep  ...     Wednesdays   

  broadcast_time                                          producers  \
0       01:00:00  Bandai Visual,Victor Entertainment,Audio Plann...   
1            NaN                              Sunrise,Bandai Visual   
2       01:15:00                               Victor Entertainment   
3       01:25:00          Bandai Visual,Dentsu,Victor Entertainment   
4       18:30:00                                    TV Tokyo,Dentsu   
5       19:00:00  TV Tokyo,Nihon Ad Systems,TV Tokyo Music,Shueisha   
6       00:35:00            Dentsu,Genco,Fuji TV,Asmik Ace,Shueisha   
7            NaN                                                NaN   
8            NaN                                        OB Planning   
9       00:40:00                      VAP,Nippon Television Network   

                                licensors           studios  \
0                              Funimation           Sunrise   
1  Sony Pictures Entertainment,Funimation             Bones   
2                              Funimation          Madhouse   
3         Funimation,Bandai Entertainment           Sunrise   
4                Illumitoon Entertainment    Toei Animation   
5              VIZ Media,Sentai Filmworks            Gallop   
6                VIZ Media,Discotek Media         J.C.Staff   
7                                     NaN  Nippon Animation   
8                              Funimation          A.C.G.T.   
9                               VIZ Media          Madhouse   

                              genres                               themes  \
0        Action,Award Winning,Sci-Fi                     Adult C

In [45]:
anime_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26541 entries, 0 to 26540
Data columns (total 30 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   title                     26541 non-null  object 
 1   english                   11870 non-null  object 
 2   japanese                  26428 non-null  object 
 3   type                      26537 non-null  object 
 4   source                    26541 non-null  object 
 5   episodes                  26144 non-null  float64
 6   status                    26541 non-null  object 
 7   aired_from                26541 non-null  object 
 8   aired_to                  10974 non-null  object 
 9   duration                  26541 non-null  object 
 10  duration_minutes          26059 non-null  float64
 11  rating                    26113 non-null  object 
 12  score                     18319 non-null  float64
 13  scored_by                 18319 non-null  float64
 14  rank  

In [46]:
anime_df["type"].unique()

array(['TV', 'Movie', 'OVA', 'TV Special', 'Special', 'ONA', 'Music',
       'CM', 'PV', nan], dtype=object)

In [ ]:

filtered_anime_df = anime_df[anime_df["type"].isin(["TV", "ONA"])]
filtered_anime_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10649 entries, 0 to 26539
Data columns (total 30 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   title                     10649 non-null  object 
 1   english                   5828 non-null   object 
 2   japanese                  10620 non-null  object 
 3   type                      10649 non-null  object 
 4   source                    10649 non-null  object 
 5   episodes                  10356 non-null  float64
 6   status                    10649 non-null  object 
 7   aired_from                10649 non-null  object 
 8   aired_to                  8136 non-null   object 
 9   duration                  10649 non-null  object 
 10  duration_minutes          10441 non-null  float64
 11  rating                    10409 non-null  object 
 12  score                     7271 non-null   float64
 13  scored_by                 7271 non-null   float64
 14  rank       

In [ ]:
import pandas as pd

def remove_null_titles(df):
    df = df[df["title"].notna() & (df["title"].str.strip() != "")]
    return df

filtered_anime_df = remove_null_titles(filtered_anime_df)
print(f"Anime con titoli validi: {len(filtered_anime_df)}")

filtered_anime_df.info()

Anime con titoli validi: 10649
<class 'pandas.core.frame.DataFrame'>
Index: 10649 entries, 0 to 26539
Data columns (total 30 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   title                     10649 non-null  object 
 1   english                   5828 non-null   object 
 2   japanese                  10620 non-null  object 
 3   type                      10649 non-null  object 
 4   source                    10649 non-null  object 
 5   episodes                  10356 non-null  float64
 6   status                    10649 non-null  object 
 7   aired_from                10649 non-null  object 
 8   aired_to                  8136 non-null   object 
 9   duration                  10649 non-null  object 
 10  duration_minutes          10441 non-null  float64
 11  rating                    10409 non-null  object 
 12  score                     7271 non-null   float64
 13  scored_by                 7271 non-

In [49]:
mangadex_titles = mangadex_df[["english", "japanese"]].fillna("").apply(lambda row: [str(row["english"]).strip().lower(), str(row["japanese"]).strip().lower()], axis=1).explode().dropna().tolist()
myanimelist_titles = filtered_anime_df[["english", "japanese"]].fillna("").apply(lambda row: [str(row["english"]).strip().lower(), str(row["japanese"]).strip().lower()], axis=1).explode().dropna().tolist()

def match_titles(row, reference_titles):
    title_list = [
        str(row["english"]).strip().lower() if pd.notna(row["english"]) else "",
        str(row["japanese"]).strip().lower() if pd.notna(row["japanese"]) else ""
    ]
    for title in title_list:
        if title and title != "nan":
            match = process.extractOne(title, reference_titles, scorer=fuzz.ratio)
            if match and match[1] > 85:
                return match[0]
    return None

filtered_anime_df["matched_title_normalized"] = anime_df.apply(match_titles, args=(mangadex_titles,), axis=1)



In [50]:
filtered_anime_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10649 entries, 0 to 26539
Data columns (total 30 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   title                     10649 non-null  object 
 1   english                   5828 non-null   object 
 2   japanese                  10620 non-null  object 
 3   type                      10649 non-null  object 
 4   source                    10649 non-null  object 
 5   episodes                  10356 non-null  float64
 6   status                    10649 non-null  object 
 7   aired_from                10649 non-null  object 
 8   aired_to                  8136 non-null   object 
 9   duration                  10649 non-null  object 
 10  duration_minutes          10441 non-null  float64
 11  rating                    10409 non-null  object 
 12  score                     7271 non-null   float64
 13  scored_by                 7271 non-null   float64
 14  rank       

In [ ]:

title_to_id_dict = {}

normalized_titles = mangadex_df["title_normalized"].dropna().str.strip().str.lower()
manga_ids = mangadex_df["manga_id"]

title_to_id_dict.update(pd.Series(manga_ids.values, index=normalized_titles).to_dict())

linguistic_columns = ["english", "japanese"]
for col in linguistic_columns:
    exploded_titles = mangadex_df.set_index("manga_id")[col].dropna().apply(lambda x: x.split("|")).explode().reset_index()
    for _, row in exploded_titles.iterrows():
        title = row[col].strip().lower()
        manga_id = row["manga_id"]
        if title:
            title_to_id_dict[title] = manga_id

filtered_anime_df["manga_id"] = anime_df["matched_title_normalized"].str.strip().str.lower().map(title_to_id_dict)

print("MyAnimeList ID unici dopo il mapping:", filtered_anime_df["manga_id"].nunique())
print("Totale matched_title_normalized:", filtered_anime_df["manga_id"].notna().sum())
filtered_anime_df.to_csv("myanimelist_matched.csv", index=False)


MyAnimeList ID unici dopo il mapping: 267
Totale matched_title_normalized: 326
